In [1]:
from qiskit import QuantumCircuit, execute
from qiskit_aer import AerSimulator
from qiskit.visualization import plot_histogram
from qiskit_aer.noise import NoiseModel, QuantumError, pauli_error
from qiskit.quantum_info import Operator, PTM, Pauli, random_quantum_channel
import numpy as np
from collections import defaultdict

In [2]:
id_gate = Operator(np.eye(4))
id_gate3 = Operator(np.eye(8))


def noisy_mcm(circ, cbit):
    """Noisy mid-circuit measurement.
    The measurement is in the measure-and-prepare fashion:
    We apply a Pauli noise, then an ideal mid-circuit measurement, followed by another Pauli noise.
    """
    circ.cnot(0, 1)
    circ.unitary(id_gate, [0, 1], label='pre')
    circ.measure(1, cbit)
    circ.unitary(id_gate, [0, 1], label='post')
    

def compiled_mcm(circ, cbit):
    """Noisy mid-circuit measurement.
    The noise is a general quantum instrument and we perform randomized compiling on it.
    """
    p = np.random.choice(['I', 'X', 'Y', 'Z'])
    alpha, beta, gamma = np.random.randint(2, size=3)
    if alpha == 0 and beta == 0:
        p12 = 'I' + p
    elif alpha == 0 and beta == 1:
        p12 = 'Z' + p
    elif alpha == 1 and beta == 0:
        p12 = 'X' + p
    else:
        p12 = 'Y' + p
    cn_circ = QuantumCircuit(2, 0)
    cn_circ.cnot(0, 1)
    p12 = Pauli(p12).evolve(cn_circ, frame='h').to_label()
    if p12[0] == '-':
        p12 = p12[1:]
    for i in range(2):
        if p12[i] == 'X':
            circ.x(1 - i)
        elif p12[i] == 'Y':
            circ.y(1 - i)
        elif p12[i] == 'Z':
            circ.z(1 - i)
    circ.reset(2)
    circ.cnot(0, 1)
    circ.unitary(id_gate3, [0, 1, 2], label='qi')
    if alpha == 1:
        circ.x(2)
    circ.measure(2, cbit)
    if p == 'X':
        circ.x(0)
    elif p == 'Y':
        circ.y(0)
    elif p == 'Z':
        circ.z(0)
    if gamma == 1:
        circ.z(1)
    if alpha == 1:
        circ.x(1)

    
def noisy_sp(circ, p):
    """Noisy state preparation.
    Though almost any initial state works for our protocol, it would be better for the initial state to big overlap with the ideal operator.
    Here we assume that we prepare the eigen state of the ideal operator and then apply a special Pauli noise (same as the terminating measurement nosie).
    """
    p = p[::-1]
    for i in range(2):
        if p[i] == 'I':
            p = p[:i] + np.random.choice(['X', 'Y', 'Z']) + p[i + 1:]
        if p[i] == 'X':
            circ.h(i)
        elif p[i] == 'Y':
            circ.h(i)
            circ.s(i)
    circ.unitary(id_gate, [0, 1], label='spam')
    

def noisy_m(circ, p, cbit):
    """Noisy terminating measurement.
    We apply a special Pauli noise and then make an ideal terminating measurement.
    If the measured operator is I, then we will still make a measurement but will ignore the output.
    """
    p = p[::-1]
    circ.unitary(id_gate, [0, 1], label='spam')
    for i in range(2):
        if p[i] == 'I':
            p = p[:i] + np.random.choice(['X', 'Y', 'Z']) + p[i + 1:]
        if p[i] == 'X':
            circ.h(i)
        elif p[i] == 'Y':
            circ.sdg(i)
            circ.h(i)
    circ.measure([0, 1], [cbit, cbit + 1])

In [3]:
def str2idx(p): # II, IX, IY, IZ, XI, XX...
    ret = 0
    for i in range(2):
        if p[i] == "X":
            ret += 1 * 4 ** (1 - i)
        elif p[i] == "Y":
            ret += 2 * 4 ** (1 - i)
        elif p[i] == "Z":
            ret += 3 * 4 ** (1 - i)
    return ret

In [4]:
def random_qi(noise_scale):
    """Generate a quantum instrument for noisy subsystem measurement.
    It is hard to satisfy complete positivity if we construct arbitrarily.
    So we have to rely on some physical structures.
    We compose noise before measurement + misreport error + noise after measurement, then average several instances to form a joint probability distribution.
    The helper channel is Q_0\otimes I+Q_1\otimes X.
    """
    tot_q0 = np.zeros([16, 16], dtype=complex)
    tot_q1 = np.zeros([16, 16], dtype=complex)
    tot_weight = 0.0
    for _ in range(8):
        p01 = np.random.rand() * noise_scale
        p10 = np.random.rand() * noise_scale
        q0 = np.kron(np.array([[1-p01+p10,0,0,1-p01-p10],[0,0,0,0],[0,0,0,0],[1-p01-p10,0,0,1-p01+p10]])/2, np.eye(4))
        q1 = np.kron(np.array([[1-p10+p01,0,0,-1+p10+p01],[0,0,0,0],[0,0,0,0],[-1+p10+p01,0,0,1-p10+p01]])/2, np.eye(4))
        n00 = PTM(random_quantum_channel(input_dims=2, output_dims=2)).data * noise_scale + np.eye(4) * (1 - noise_scale)
        n01 = PTM(random_quantum_channel(input_dims=2, output_dims=2)).data * noise_scale + np.eye(4) * (1 - noise_scale)
        n10 = PTM(random_quantum_channel(input_dims=2, output_dims=2)).data * noise_scale + np.eye(4) * (1 - noise_scale)
        n11 = PTM(random_quantum_channel(input_dims=2, output_dims=2)).data * noise_scale + np.eye(4) * (1 - noise_scale)
        weight = np.random.rand()
        tot_weight += weight
        tot_q0 += np.kron(n00, n01) @ q0 @ np.kron(n10, n11) * weight
        tot_q1 += np.kron(n00, n01) @ q1 @ np.kron(n10, n11) * weight
    tot_q0 /= tot_weight
    tot_q1 /= tot_weight
    helper_chan = PTM(np.kron(np.eye(4), tot_q0) + np.kron(np.diag([1,1,-1,-1]), tot_q1))
    assert helper_chan.is_cptp()
    # now we have the instrument, calculate the error rates and fidelities
    rate = {}
    fid = defaultdict(float)
    channels = [tot_q0, tot_q1]
    p2 = ['I', 'Z']
    p4 = ['I', 'X', 'Y', 'Z']
    for a in range(2):
        for b in range(2):
            Lab = np.zeros(4)
            for x in range(2):
                for A in range(2):
                    for B in range(4):
                        for C in range(2):
                            Lab[B] += np.real_if_close((-1) ** ((b + x) * A + (a + x) * C) * channels[x][str2idx(p2[A] + p4[B]), str2idx(p2[C] + p4[B])]) / 4
            rate['I' + str(a) + str(b)] = (Lab[0] + Lab[1] + Lab[2] + Lab[3]) / 4
            rate['X' + str(a) + str(b)] = (Lab[0] + Lab[1] - Lab[2] - Lab[3]) / 4
            rate['Y' + str(a) + str(b)] = (Lab[0] - Lab[1] + Lab[2] - Lab[3]) / 4
            rate['Z' + str(a) + str(b)] = (Lab[0] - Lab[1] - Lab[2] + Lab[3]) / 4
            for B in range(4):
                for c in range(2):
                    for d in range(2):
                        fid[p4[B] + str(c) + str(d)] += Lab[B] * (-1) ** (a * c + b * d)
    return helper_chan, fid, rate

In [5]:
"""
The noise model.
"""

# Noises for measure-and-prepare mid-circuit measurement. Noises are Pauli channels.
rate_pre = {}
for i in ["I", "X", "Y", "Z"]:
    for j in ["I", "X", "Y", "Z"]:
        rate_pre[i+j] = np.random.rand() * 0.01 ** ((i != 'I') + (j != 'I'))
rate_pre["II"] += 1 - np.sum([v for v in rate_pre.values()])
err_pre = pauli_error(list(rate_pre.items()))
fid_pre = np.real_if_close(np.diag(PTM(err_pre).data))

rate_post = {}
for i in ["I", "X", "Y", "Z"]:
    for j in ["I", "X", "Y", "Z"]:
        rate_post[i+j] = np.random.rand() * 0.02 ** ((i != 'I') + (j != 'I'))
rate_post["II"] += 1 - np.sum([v for v in rate_post.values()])
err_post = pauli_error(list(rate_post.items()))
fid_post = np.real_if_close(np.diag(PTM(err_post).data))

"""
fid_refresh = {}
for i in ["I", "X", "Y", "Z"]:
    for j in ["0", "1"]:
        for k in ["0", "1"]:
            _, s, t = fid2st(i + j + k)
            fid_refresh[i + j + k] = fid_pre[str2idx(s)] * fid_post[str2idx(t)]
"""

# Noises for spam error. Since state preparation error does not matter in our theory, we set SPAM error the same for simplicity.
# Terminating measurement errors is a special kind of Pauli channel.
rate_spam = {}
for i in [["I"], ["X", "Y", "Z"]]:
    for j in [["I"], ["X", "Y", "Z"]]:
        rate = (1 + np.random.rand()) * 0.015 ** ((len(i) + len(j)) / 2 - 1)
        for ii in i:
            for jj in j:
                rate_spam[ii + jj] = rate
rate_spam["II"] += 1 - np.sum([v for v in rate_spam.values()])
err_spam = pauli_error(list(rate_spam.items()))

# General noisy quantum instrument. Need to be compiled.
err_qi, fid_qi, rate_qi = random_qi(0.002)

# Put the noises into the simulator.
noise_model = NoiseModel()
noise_model.add_basis_gates(['unitary'])
noise_model.add_all_qubit_quantum_error(err_pre, 'pre')
noise_model.add_all_qubit_quantum_error(err_post, 'post')
noise_model.add_all_qubit_quantum_error(err_spam, 'spam')
noise_model.add_all_qubit_quantum_error(err_qi, 'qi')
backend_noisy = AerSimulator(noise_model=noise_model)

In [6]:
def fid2st(fid):
    s = fid[0]
    t = fid[0]
    if fid[1] == '0':
        s = 'I' + s
    else:
        s = 'Z' + s
    if fid[2] == '0':
        t = 'I' + t
    else:
        t = 'Z' + t
    cn_circ = QuantumCircuit(2, 0)
    cn_circ.cnot(0, 1)
    s = Pauli(s).evolve(cn_circ, frame='h').to_label()
    sgn = 1
    if s[0] == '-':
        sgn *= -1
        s = s[1:]
    return sgn, s, t

In [7]:
def learn_single_fid(fid, shots=10000, m_p=False):
    """Learn e+\partial e"""
    sgn, s, t = fid2st(fid)
    s_avg = 0
    for _ in range(100):
        circ = QuantumCircuit(3, 3)
        noisy_sp(circ, s)
        if m_p:
            noisy_mcm(circ, 0)
        else:
            compiled_mcm(circ, 0)
        noisy_m(circ, t, 1)
        counts = execute(circ, backend_noisy, shots=int(shots / 100)).result().get_counts()
        for res in counts:
            count = counts[res]
            res_par = np.array([int(c) for c in res])[::-1]
            for i in range(2):
                if t[i] == 'I':
                    res_par[2 - i] = 0
            s_avg += sgn * count * (-1) ** (res_par[0] * (int(fid[1]) + int(fid[2])) + res_par[1] + res_par[2])
    s_avg /= (int(shots / 100) * 100)
    
    circ2 = QuantumCircuit(2, 2)
    noisy_sp(circ2, s)
    noisy_m(circ2, s, 0)
    counts = execute(circ2, backend_noisy, shots=shots).result().get_counts()
    t_avg = 0
    for res in counts:
        count = counts[res]
        res_par = np.array([int(c) for c in res])[::-1]
        for i in range(2):
            if s[i] == 'I':
                res_par[1 - i] = 0
        t_avg += count * (-1) ** (res_par[0] + res_par[1])
    t_avg /= shots
    return np.log(s_avg / t_avg)

In [16]:
learned_fid = {}
for i in ["I", "X", "Y", "Z"]:
    for j in ["0", "1"]:
        for k in ["0", "1"]:
            learned_fid[i + j + k] = learn_single_fid(i + j + k, shots=1000000, m_p=False)

In [10]:
# self loop is learnable
print(np.exp(learned_fid["Y01"]))
print(fid_qi["Y01"])

0.9862100728780039
0.9920924281713054


In [17]:
# cycle is learnable
print(np.exp(learned_fid["Z01"] + learned_fid["X00"]))
print(fid_qi["Z01"] * fid_qi["X00"])

0.987508250720592
0.9883210015278241


In [18]:
print(np.exp(learned_fid["I01"] + learned_fid["Z10"] + learned_fid["Z01"] + learned_fid["I10"]))
print(fid_qi["I01"] * fid_qi["Z10"] * fid_qi["Z01"] * fid_qi["I10"])

0.9781101856274068
0.9780066298274457


In [19]:
# simple edge is not learnable
print(np.exp(learned_fid["Z01"]))
print(fid_qi["Z01"])

0.9217196224875873
0.9923239966504842


In [21]:
# Testing the independence of measurement and state preparation.
print((learned_fid["Z00"] + learned_fid["Z11"]) - (learned_fid["Z01"] + learned_fid["Z10"]))
# Though the result is small, its deviation from 0 is greater than standard deviation.
print(np.log(fid_qi["Z00"] * fid_qi["Z11"]) - np.log(fid_qi["Z01"] * fid_qi["Z10"]))

0.0038040322214518
0.003251117035598959


In [22]:
learned_fid = {}
for i in ["I", "X", "Y", "Z"]:
    for j in ["0", "1"]:
        for k in ["0", "1"]:
            learned_fid[i + j + k] = learn_single_fid(i + j + k, shots=1000000, m_p=True)

In [23]:
# Testing the independence of measurement and state preparation.
learned_fid["Z00"]+learned_fid["Z11"]-learned_fid["Z01"]-learned_fid["Z10"]
# This time the result is within standard deviation

0.001639604875861872

In [24]:
def noiseless_concat(circ, t, s):
    """Append the single qubit Clifford gates."""
    for i in range(2):
        if t[i] != s[i]:
            if t[i] == 'I' or s[i] == 'I':
                raise ValueError('Mismatched Pauli pattern.')
            if t[i] == 'X':
                if s[i] == 'Y':
                    circ.s(1 - i)
                else:
                    circ.h(1 - i)
            elif t[i] == 'Y':
                if s[i] == 'Z':
                    circ.sdg(1 - i)
                    circ.h(1 - i)
                else:
                    circ.sdg(1 - i)
            else:
                if s[i] == 'X':
                    circ.h(1 - i)
                else:
                    circ.h(1 - i)
                    circ.s(1 - i)

In [25]:
def learn_multiple_fid(fids, shots=10000, m_p=False): # add logic for m_p
    """Learn a path"""
    if m_p:
        mcm = noisy_mcm
    else:
        mcm = compiled_mcm
    l = len(fids)
    ss = [""] * l
    ts = [""] * l
    tot_sgn = 1
    for i in range(l):
        sgn, ss[i], ts[i] = fid2st(fids[i])
        tot_sgn *= sgn
    s_avg = 0
    for _ in range(100):
        circ = QuantumCircuit(3, l + 2)
        noisy_sp(circ, ss[0])
        for i in range(l - 1):
            mcm(circ, i)
            noiseless_concat(circ, ts[i], ss[i + 1])
        mcm(circ, l - 1)
        noisy_m(circ, ts[l - 1], l)
        counts = execute(circ, backend_noisy, shots=int(shots / 100)).result().get_counts()
        for res in counts:
            count = counts[res]
            res_par = np.array([int(c) for c in res])[::-1]
            for i in range(2):
                if ts[l - 1][i] == 'I':
                    res_par[l + 1 - i] = 0
            mxy = 0
            for i in range(l):
                mxy += res_par[i] * (int(fids[i][1]) + int(fids[i][2]))
            s_avg += tot_sgn * count * (-1) ** (mxy + res_par[l] + res_par[l + 1])
    s_avg /= (int(shots / 100) * 100)
    
    circ2 = QuantumCircuit(2, 2)
    noisy_sp(circ2, ss[0])
    noisy_m(circ2, ss[0], 0)
    counts = execute(circ2, backend_noisy, shots=shots).result().get_counts()
    t_avg = 0
    for res in counts:
        count = counts[res]
        res_par = np.array([int(c) for c in res])[::-1]
        for i in range(2):
            if ss[0][i] == 'I':
                res_par[1 - i] = 0
        t_avg += count * (-1) ** (res_par[0] + res_par[1])
    t_avg /= shots
    return np.log(s_avg / t_avg)

In [26]:
targ = ['X11','Y11', 'Y00', 'Z01', 'X00', 'Z00', 'Z01', 'I11', 'Z11']
learned_res = learn_multiple_fid(targ, shots=100000, m_p=False) 
print(np.exp(learned_res)) # learned value of this product
print(np.prod([fid_qi[i] for i in targ])) # true value of this product
targ = ['Y01','X10', 'Z01', 'X01', 'Y10', 'Z01', 'I10', 'I01', 'Z10', 'Z01']
learned_res -= learn_multiple_fid(targ, shots=100000, m_p=False)
print(learned_res / 16) # learned p^I_{11}

0.9449461429659448
0.9460714913861634
0.0007311721345428946


In [28]:
print(rate_qi['I11']) # true value of p^I_{11}

0.0008088998585483384


In [29]:
pI11_apx = 0.
for k, v in fid_qi.items():
    pI11_apx += (1 + np.log(v)) * (-1) ** (int(k[1]) + int(k[2]))
pI11_apx /= 16
print(pI11_apx) # p^I_{11} under first order approximation=0

0.0008126760418828219


In [30]:
targ = ['I00', 'I01', 'Z11', 'I11', 'Z10', 'Z00', 'Z01', 'I10']
print(learn_multiple_fid(targ, shots=100000, m_p=False) / 8 + 1) # Learned value of p^I_{00} + p^Z_{00}
print(rate_qi['I00'] + rate_qi['Z00']) # True value of p^I_{00} + p^Z_{00}

0.995227543094729
0.9952675190723601


In [31]:
pI00Z00_apx = 0
for k, v in fid_qi.items():
    if k[0] == 'I' or k[0] == 'Z':
        pI00Z00_apx += 1 + np.log(v)
pI00Z00_apx /= 8
print(pI00Z00_apx) # p^I_{00} + p^Z_{00} under first order approximation

0.9952530241138203
